In [ ]:
#******************************************************
#*
#* Name:         nb-01-manage-zip-files
#*     
#* Design Phase:
#*     Author:   John Miner
#*     Date:     12-04-2024
#*     Purpose:  Unzip and zip data files.
#* 
#******************************************************/

In [ ]:
#
#  Suggest zipping character based files (csv, json, text)
#

In [2]:
#
#  1 - reset unzip demo - delete 2 files
#


# remove csv file
path1 = "file:/lakehouse/default/Files/Stocks/all_stock_data.csv"
try:
    mssparkutils.fs.rm(path1) 
except:
    pass


# remove parquet file
path2 = "file:/lakehouse/default/Files/Stocks/all_stock_data.parquet"
try:
    mssparkutils.fs.rm(path2) 
except:
    passs


StatementMeta(, 1a3593e6-0f65-4d87-9c7b-51cec1f9a708, 4, Finished, Available, Finished)

In [3]:

#
#  2 - unzip files 
#

# import library
import zipfile as z

# set file + path
path3 = "/lakehouse/default/Files/Stocks/"
file3 = path3 + "archive.zip"

# extract all files
with z.ZipFile(file3) as myzip:
    myzip.extractall(path3)


StatementMeta(, 1a3593e6-0f65-4d87-9c7b-51cec1f9a708, 5, Finished, Available, Finished)

In [1]:
#
#  3 - show file sizes
#

# extract 1.9 GB zip file into 3.5 GB csv file + 0.96 GB parquet file

path4 = "file:/lakehouse/default/Files/Stocks/"
mssparkutils.fs.ls(path4)


StatementMeta(, d2b33464-3908-4eda-a64b-759c29b34144, 3, Finished, Available, Finished)

[FileInfo(path=file:/lakehouse/default/Files/Stocks/all_stock_data.csv, name=all_stock_data.csv, size=3512208588),
 FileInfo(path=file:/lakehouse/default/Files/Stocks/all_stock_data.parquet, name=all_stock_data.parquet, size=962269317),
 FileInfo(path=file:/lakehouse/default/Files/Stocks/archive.zip, name=archive.zip, size=1918054636)]

In [43]:
#
#  4 - virtual mounts supported only by extact method, not write method
#

mssparkutils.fs.mounts()

StatementMeta(, 1a3593e6-0f65-4d87-9c7b-51cec1f9a708, 45, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c82aba33-1064-429d-a6f6-9708bb0b6d18)

[MountPointInfo(mountPoint=/default, source=abfss://a668a328-9f67-4678-93f2-10d5afdfe3ad@onelake.dfs.fabric.microsoft.com/1bc3c138-54ea-4c7e-b759-03edb4673eba, scope=default_lh, localPath=/lakehouse/default),
 MountPointInfo(mountPoint=/nb_resource/builtin, source=Notebook Working Directory, scope=nb_resource, localPath=/synfs/nb_resource/builtin)]

In [3]:
#
#  5 - Define function to find matching files
# 

# import libraries
import fnmatch

# define function
def get_file_list(path_txt, pattern_txt):
  
  # list of file info objects
  fs_lst = mssparkutils.fs.ls(path_txt)
  
  # create list of file names
  dir_lst = list()
  for f in fs_lst:
      dir_lst.append(f.name)
      
  # filter file names by pattern
  files_lst = fnmatch.filter(dir_lst, pattern_txt)
  
  # return list
  return(files_lst)

StatementMeta(, bd240364-984f-475c-8750-d0533b5f521e, 5, Finished, Available, Finished)

In [4]:
#
#  6 - test function
#

get_file_list('Files/Msft', '*.CSV')

StatementMeta(, bd240364-984f-475c-8750-d0533b5f521e, 6, Finished, Available, Finished)

['MSFT-FY2013.CSV',
 'MSFT-FY2014.CSV',
 'MSFT-FY2015.CSV',
 'MSFT-FY2016.CSV',
 'MSFT-FY2017.CSV']

In [21]:
#
#  7 - Define function to zip up files
# 

import io
import zipfile as z

# define function
def zip_up_files(folder, pattern, archive_file):

    # set variables
    path1 = f"Files/{folder}/"
    path2 = f"/lakehouse/default/Files/{folder}/"
    pat1 = f"*.{pattern}"

    # get list of files
    files = get_file_list(path1, pat1)    

    # define memory buffer
    zip_buffer = io.BytesIO()

    # read files into memory buffer
    with z.ZipFile(zip_buffer, "a", z.ZIP_DEFLATED, False) as zip_file:
        for file in files:
            fh = open(path2 + file, "rb")
            str = io.BytesIO(fh.read()).getvalue()
            zip_file.writestr(file, str)

    # write out buffer to virtual disk
    with open(path2 + f'{archive_file}', 'wb') as f:
        f.write(zip_buffer.getvalue())


StatementMeta(, bd240364-984f-475c-8750-d0533b5f521e, 23, Finished, Available, Finished)

In [22]:

#
#  7 - test function
#

zip_up_files('Msft', 'CSV', 'archive2.zip')

StatementMeta(, bd240364-984f-475c-8750-d0533b5f521e, 24, Finished, Available, Finished)